In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 



# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
  #  for filename in filenames:
   #     print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Recommendation System

we use cosine similarity to predict a title. Cosine similarity takes the angle between two non-zero vectors and calculates the cosine of that angle, and this value is known as the similarity between the two vectors. This similarity score ranges from 0 to 1, with 0 being the lowest (the least similar) and 1 being the highest (the most similar).

In [ ]:
!pip install rake_nltk

In [ ]:
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
df = pd.read_csv('/kaggle/input/netflix-shows/netflix_titles.csv')

this recommendation system is based on similarity between type, listed_in(genre), rating, director, cast and description between titles.

In [ ]:
df = df[['title','type','listed_in','rating','director','cast','description']]
df.head()

Everything needs to be lowercase to avoid duplications, and I also decided to merge all first and last names in one unique word. Because, think about it: if we have movie A where the director is Danny Boyle, and movie B where one of the main actors is Danny DeVito, the recommender will detect a similarity because of their first name and that is something we don’t want. I want the recommender to detect a similarity only if the person associated to different movies is exactly the same.
After all the cleaning and merging, I reassigned the index to the movie title column (which is not among my features) and this is the dataframe ready for vectorization.

In [ ]:
#converting the entire description column into a list
#where every word is an element and renaming the description column into bag_of_words.
df['Key_words'] = ""

for index, row in df.iterrows():
    plot = row['description']
    r = Rake()
    r.extract_keywords_from_text(plot)
    key_words_dict_scores = r.get_word_degrees()
    #a rating is also assigned to every word
    row['Key_words'] = list(key_words_dict_scores.keys())
    
df.drop(columns = ['description'], inplace = True)
df.head()

In [ ]:
df.isnull().sum()

we need to replace all the null values with ''

In [ ]:
df['cast'] = df['cast'].fillna('')
df['director'] = df['director'].fillna('')
df['rating'] = df['rating'].fillna('')

df['cast'] = df['cast'].map(lambda x: x.split(',')[:3])
df['director'] = df['director'].map(lambda x: x.split(',')[:3])

In [ ]:
df['listed_in'] = df['listed_in'].map(lambda x: x.lower().split(','))
df.head()

In [ ]:
#removing all spaces between words and names
#convering all the words into lower case
for index, row in df.iterrows():
    row['cast'] = [x.lower().replace(' ','') for x in row['cast']]
for index, row in df.iterrows():
    row['director'] = [x.lower().replace(' ','') for x in row['director']]
for index, row in df.iterrows():
    row['listed_in'] = [x.lower().replace(' ','') for x in row['listed_in']]

In [ ]:
#all the columns, recommendation is based on, should be all lists
df['rating'] = df['rating'].map(lambda x: x.lower())
df['rating'] = df['rating'].map(lambda x: x.replace('-',''))
df['type'] = df['type'].map(lambda x: x.lower())
df['type'] = df['type'].map(lambda x: x.replace(' ',''))
df['type'] = df['type'].map(lambda x: x.lower().split(','))
df['rating'] = df['rating'].map(lambda x: x.lower().split(','))

In [ ]:
df.set_index('title', inplace = True)
df.head()

In [ ]:
#creating a bag_of_words
#combining all the lists into a single list
df['bag_of_words'] = ''
columns = df.columns
for index, row in df.iterrows():
    words = ''
    for col in columns:
        words = words + ' '.join(row[col])+ ' '
    row['bag_of_words'] = words
    
df.drop(columns = [col for col in df.columns if col!= 'bag_of_words'], inplace = True)

In [ ]:
df.head()

In [ ]:
count = CountVectorizer()
count_matrix = count.fit_transform(df['bag_of_words'])

indices = pd.Series(df.index)
indices[:5]

In [ ]:
# generating the cosine similarity matrix
# comparing bag_of_words of every title with every other title creating nxn matrix where n is total number of rows(titles)
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

In [ ]:
def recommendations(title, cosine_sim = cosine_sim):
    recommended_movies = []
    idx = indices[indices == title].index[0]
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    top_10_indexes = list(score_series.iloc[1:11].index)
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
        
    return recommended_movies

In [ ]:
# Time for some recommendations
recommendations('Naruto Shippuden : Blood Prison')

In [ ]:
recommendations('PK')

the system is giving some good quality recommendations
looks like our basic recommendation system is working as we wanted :)